# Notebook to analyze HIV Dataset #

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '/home/runner/MEGA/Repo/giotto-learn/giotto/graphs')

In [ ]:
#Import statements

import numpy as np
import networkx as nx
import scipy as sp

import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import random 
import pandas as pd

from create_clique_complex import CreateCliqueComplex, CreateBoundaryMatrices, CreateLaplacianMatrices
from heat_diffusion import HeatDiffusion
from graph_entropy import GraphEntropy
from matplotlib.pyplot import pause

from molecules import mol_to_nx, compute_node_edge_entropy
from utils.plotting import plot_network_diffusion

from utils.plotting import plot_entropies, plot_network_diffusion

import pickle

import deepchem as dc
from rdkit import Chem 
from rdkit.Chem import Draw

import random


## Import and Convert data to networkx Graph ##

In [ ]:
#Import as csv for the method

df = pd.read_csv('hiv.csv')

## Convert data ##

In [ ]:
#Method that transforms smiles strings to mol rdkit object
df['mol'] = df['smiles'].apply(lambda x: Chem.MolFromSmiles(x)) 



In [ ]:
#Plot example of molecule 0
mol = df['mol'][0]
g = mol_to_nx(mol)
pos = nx.spring_layout(g)

nx.draw(g, pos)

mols = [mol]
Draw.MolsToGridImage(mols, molsPerRow=5, useSVG=False)

In [ ]:
#Convert all molecules and put them into a list g_mol
g_mol = df['mol'].apply(lambda x: mol_to_nx(x))


In [ ]:
#Poor 'class 1' example
print('Number of active inhibitors: {}'.format(df[df['HIV_active'] == 1].shape[0]))


In [ ]:
#Hyperparameters 


#Temporal instants to take for node diffusions
taus_n = np.linspace(0, 2, 20)


#Temporal instants to take for edge diffusions
taus_e = np.linspace(0, 2, 20)




In [ ]:
g = g_mol[0]

In [ ]:
ci = np.zeros((19, 2))

In [ ]:
ci[0,0] = 1
ci[1,1]=1

In [ ]:
ci.shape

In [ ]:
cd = CreateCliqueComplex(graph=g).create_complex_from_graph()
lap = CreateLaplacianMatrices().fit().transform(cd, (0,1) )
mh_n = GraphEntropy().fit().transform(HeatDiffusion().fit(taus_n).transform(lap[0])).T
mh_e = GraphEntropy().fit().transform(HeatDiffusion().fit(taus_e).transform(lap[1])).T

In [ ]:
mh_n.shape

In [ ]:
plot_entropies(mh_e, [0,3,4,5,6, 15])

In [ ]:
#Simple K-mean
kmeans = KMeans(n_clusters=4)
kmeans.fit(np.transpose(mh_n))
y_kmeans_n = kmeans.predict(np.transpose(mh_n))

In [ ]:
#Just for Visualization, plot 2D PCA embedding with points colored by classes


pca = PCA(n_components=4)

pca_n = pca.fit_transform(np.transpose(mh_n))

#Colors for classes (at most 8)
col = ['blue', 'red', 'black', 'yellow', 'orange', 'green', 'grey', 'brown']


cols = [col[e] for e in y_kmeans_n]
plt.scatter(np.ravel(pca_n[:,0]),np.ravel(pca_n[:,1]),  c= cols, s=100)
_ = plt.title("2D-PCA of Nodes Molecule")
plt.savefig("Molecules_Node.png")

In [ ]:
pos = nx.spring_layout(g, iterations=2000)

In [ ]:
#Plot the graph with nodes colored by classes

plot_network_diffusion(g,pos,node_vector=y_kmeans_n, edge_vector=None,node_labels=True,edge_labels=False)




In [ ]:
#Simple K-mean
kmeans = KMeans(n_clusters=3)
kmeans.fit(np.transpose(mh_e))
y_kmeans_e = kmeans.predict(np.transpose(mh_e))

In [ ]:
#Just for Visualization
pca = PCA(n_components=4)

pca_e = pca.fit_transform(np.transpose(mh_e))

#Colors for classes (at most 8)
col = ['blue', 'red', 'black', 'yellow', 'orange', 'green', 'grey', 'brown']

#Associate colors to each edge w.r.t. its class
cols = [col[e] for e in y_kmeans_e]
plt.scatter(np.ravel(pca_e[:,0]),np.ravel(pca_e[:,1]),  c= cols, s=100)

_ = plt.title("2D-PCA of Edge Molecule")
plt.savefig("Molecules_Edge.png")


In [ ]:
pos = nx.spring_layout(g, iterations=2000)

In [ ]:
#Plot the graph with edges colored by classes

plot_network_diffusion(g,pos, edge_vector=y_kmeans_e,node_labels=True,edge_labels=False)




## Create Embedding for all molecules of the dataset ##

In [ ]:
embeds = [embedd(x,i, taus_n, taus_e) for i,x in enumerate(g_mol) ]


In [ ]:
nx.draw(g_mol[559])

In [ ]:
g_mol.pop(8097)
g_mol.pop(559)

In [ ]:
for i,e in enumerate(g_mol):
    if e.number_of_nodes() == 2:
        print(i)

## Save ##

In [ ]:
#Save all the Embeddings 

#pickle.dump(embeds, open("hiv_embeddings.pickle", "wb"))

## Load ##

In [ ]:
#Load it and look at the dimensions 

#embeds = pickle.load(open("hiv_embeddings.pickle", 'rb'))
print("Dataset loaded...")
print('Size of embeddings: {}'.format(embeds[0][0].shape))

In [ ]:
#Study how many different atoms (and which)

freq = dict()
for g in g_mol:
    l = list(nx.get_node_attributes(g, 'atomic_num').values())
    for a in l:
        if (freq.get(a)):
            freq[a] += 1
        else:
            freq[a]=1


In [ ]:
#Plot results
plt.scatter(freq.keys(), freq.values())
plt.title("Frequency of atom types")
plt.xlabel("Element Atomic Numbers")
plt.ylabel("Frequency in all molecules")

# Adding Bonds information #

In [ ]:
#Check which bonds we have in the dataset

s = set()
for g in g_mol:
    for n in g.nodes():
        for i in  g.neighbors(n):
           s.add(int(g.get_edge_data(n,i)['bond_type']))
print(s)

In [ ]:
#Complex Molecule
Draw.MolsToGridImage(df['mol'][3614:3615], molsPerRow=5, useSVG=False)

In [ ]:
#Plot networkx molecule

pos = nx.spring_layout(g_mol[3614], iterations=2000)
plot_network_diffusion(g_mol[3614],pos, node_labels=True)

In [ ]:
#Addingind the bond types as features

for g in g_mol:
    #dictionary for nodes
    d_n = dict()
    
    for n in g.nodes():
        d_n[n] = np.zeros(4)
        
        for i in g.neighbors(n):
            #encoding type
            edge_type = int(g.get_edge_data(n,i)['bond_type'])
            if (edge_type == 12):
                d_n[n][3] += 1
            else:
                d_n[n][edge_type-1] += 1
    #Adding bonds information to each node of all molecules
    nx.set_node_attributes(g, name='bonds_one_hot', values=d_n)
        


In [ ]:
#Create a list with all nodes 
freq_type_bonds = list()
for g in g_mol:
    freq_type_bonds.extend(list(nx.get_node_attributes(g, 'bonds_one_hot').values()))

In [ ]:
freq_type_bonds = np.array(freq_type_bonds)
print("Total number of atoms in the dataset: {}".format(freq_type_bonds.shape[0]))
print("Entry size: {}".format(freq_type_bonds.shape[1]))

# Universal Node Embeddding #

In [ ]:
#Universal embedding for nodes 
embedding_dim = 40

universal_points = list()
for e in embeds:
    #Iterate over all nodes
    for n in range(embeds[e][0].shape[1]):
        universal_points.append(np.ravel(embeds[e][0][1:,n]))

universal_points = np.array(universal_points)
universal_points.shape

In [ ]:
uni_frq = list()
for x in range(universal_points.shape[0]):
    uni_frq.append(np.hstack([universal_points[x,:], freq_type_bonds[x,:]]))

In [ ]:
uni_frq = np.array(uni_frq)
uni_frq.shape

In [ ]:
s = 0
for e in embeds:
    s+= embeds[e][0].shape[1]

In [ ]:
j=0
graph_to_points = dict()
for i in range(len(g_mol)):
    #if (g_mol[i].number_of_nodes() > 10 and g_mol[i].number_of_nodes() < 100):
    graph_to_points[i] = np.arange(j, j + g_mol[i].number_of_nodes())
    j += g_mol[i].number_of_nodes()

In [ ]:
#Use only topological information

#Simple K-mean

kmeans_n = KMeans(n_clusters=10)
kmeans_n.fit(uni_frq)
universal_class = kmeans_n.predict(uni_frq)

In [ ]:
centroids_n = kmeans_n.cluster_centers_

In [ ]:
"""from sklearn import metrics

metrics.silhouette_score(uni_frq, universal_class, metric='euclidean')"""

In [ ]:
#Just for Visualization, plot 2D PCA embedding with points colored by classes


pca = PCA(n_components=2)

pca_n = pca.fit_transform(universal_points)

#Colors for classes (at most 8)
col = ['blue', 'red', 'black', 'yellow', 'orange', 'green', 'grey', 'brown', 'violet', 'yellow']


cols = [col[e] for e in universal_class]
plt.scatter(np.ravel(pca_n[:,0]),np.ravel(pca_n[:,1]),  c= cols, s=100)
_ = plt.title("2D-PCA of Node Emebdding")


# First Topology approach #

In [ ]:
#Bonds info (one_hot_encoding)
bn = dict()
#Nodes sums
ran = dict()
#Edges sum
rane = dict()
for e in embeds:  
    
    print(e)
    if (g_mol[e].number_of_nodes() < 100 and g_mol[e].number_of_nodes() >10):
        ran[e] = list(sum(embeds[e][0]))
        ran[e] = list( ran[e] / np.max(ran[e]))
        #ran[e] = list( ran[e])

        rane[e] = list(sum(embeds[e][1]))
        rane[e] = list( rane[e] / np.max(rane[e]))
        #rane[e] = list(rane[e])

        #for a in nx.get_node_attributes(g_mol[e], 'atomic_num').values():
        #        at[e][mapp_atoms[a]] +=1
        
        
    

# Universal Edge Emebdding #

In [ ]:
#Universal embedding for nodes 
embedding_dim = 40

universal_edge = list()
for e in embeds:
    #Iterate over all nodes
    for n in range(embeds[e][1].shape[1]):
        universal_edge.append(np.ravel(embeds[e][1][1:,n]))

universal_edge = np.array(universal_edge)
universal_edge.shape

In [ ]:
for g in g_mol:
    d_e = dict()
    for e in g.edges():
        d_e[e] = np.zeros(4)
        if (b == 12):
            d_e[e][3] = 1
        else:
            d_e[e][b-1] = 1
    nx.set_edge_attributes(g, name='bonds_one_hot', values=d_e)

In [ ]:
freq_type_bonds_edge = list()
for g in g_mol:
    freq_type_bonds_edge.extend(list(nx.get_edge_attributes(g, 'bonds_one_hot').values()))

In [ ]:
freq_type_bonds_edge = np.array(freq_type_bonds_edge)
freq_type_bonds_edge.shape

In [ ]:
uni_frq_edge = list()
for x in range(universal_edge.shape[0]):
    uni_frq_edge.append(np.hstack([universal_edge[x,:], freq_type_bonds_edge[x,:]]))

In [ ]:
uni_frq_edge = np.array(uni_frq_edge)

In [ ]:
uni_frq_edge.shape

In [ ]:
#Use only Topological information

#Simple K-mean

kmeans_e = KMeans(n_clusters=10)
kmeans_e.fit(uni_frq_edge)
universal_class_edge = kmeans_e.predict(uni_frq_edge)

centroids_e = kmeans_e.cluster_centers_

In [ ]:
#Just for Visualization, plot 2D PCA embedding with points colored by classes


pca = PCA(n_components=2)

pca_n = pca.fit_transform(uni_frq_edge)

#Colors for classes (at most 8)
col = ['blue', 'red', 'black', 'yellow', 'orange', 'green', 'grey', 'brown', 'violet', 'yellow']


cols = [col[e] for e in universal_class_edge]
plt.scatter(np.ravel(pca_n[:,0]),np.ravel(pca_n[:,1]),  c= cols, s=100)
_ = plt.title("2D-PCA of Edge Emebdding")



In [ ]:
j=0
graph_to_edges = dict()
for i in range(len(g_mol)):
    #if (g_mol[i].number_of_nodes() > 10 and g_mol[i].number_of_nodes() < 100):
    graph_to_edges[i] = np.arange(j, j + g_mol[i].number_of_edges())
    j += g_mol[i].number_of_edges()

In [ ]:
b = np.linspace(60,160, 50)

In [ ]:
h, f, _ =  plt.hist(ran[1234], bins=b)

In [ ]:
h, f, _ =  plt.hist(rane[1234], bins=b)

In [ ]:
#Molecule sizes frequency

dims = [t.number_of_nodes() for t in g_mol]
_ = plt.hist(dims, bins=100)
_ = plt.title("Molecule sizes frequency")

In [ ]:
b = np.linspace(0.4,1, 10)

In [ ]:
#Put toghether information of first approach

x = list()
for i in ran:
    l = list()
    h, _ = np.histogram(ran[i], bins=b)
    h = list(h)
    h.append(g_mol[i].number_of_nodes())
    l = h

    he, _ = np.histogram(rane[i], bins=b)
    he = list(he)
    he.append(g_mol[i].number_of_edges())
    l.extend(he)
    
    #l.extend(at[i])
    x.append(l)
    
    

In [ ]:
a = np.array(x)

In [ ]:
a.shape

In [ ]:
pickle.dump(a, open("x_data.pickle", "wb"))

In [ ]:
x_data = a

In [ ]:
x_data[0]

In [ ]:
x_data.shape

# Data with universal embedding #


### Nodes ###

In [ ]:
uni_frq.shape

In [ ]:
universal_class.shape

In [ ]:
one_hot_encoded = np.zeros((universal_points.shape[0], 10))
for x in range(len(universal_points)):
    one_hot_encoded[x][universal_class[x]] = 1

In [ ]:
#Soft Encoded
soft_encoded_node = np.zeros((uni_frq.shape[0], 10))
for x in range(uni_frq.shape[0]):
    for c in range(10):
        soft_encoded_node[x,c] = np.exp( - (np.linalg.norm(uni_frq[x]- centroids_n[c], 2) ** 2) / 2)

In [ ]:
#Normalize
for x in soft_encoded_node:
    x /= np.sum(x)

In [ ]:
soft_encoded_node[1023]

In [ ]:
one_hot_encoded[0]

In [ ]:
plt.scatter(range(10), np.sum(one_hot_encoded, axis=0))

In [ ]:
print(np.sum(one_hot_encoded, axis=0))

In [ ]:
type(g_mol)

In [ ]:
one_hot_encoded.shape

In [ ]:
freq_type_bonds[5].shape

In [ ]:
#Create data for each graph
x_data_node = list()
for i in range(len(g_mol)):
    if (g_mol[i].number_of_nodes() > 10 and g_mol[i].number_of_nodes() < 100):
        v = np.zeros((soft_encoded_node[0].shape[0]))
        #v = np.zeros((one_hot_encoded[0].shape[0]))
        for n in graph_to_points[i]:
            v += soft_encoded_node[n]
            #v += one_hot_encoded[n]
        #x_data.append(np.hstack([v,t]))
        x_data_node.append(v)
x_data_node = np.array(x_data_node)
x_data_node.shape

### Edges ###

In [ ]:
universal_class_edge.shape[0]

In [ ]:
one_hot_encoded_edge = np.zeros((universal_class_edge.shape[0], 10))
for x in range(len(universal_edge)):
    one_hot_encoded_edge[x][universal_class_edge[x]] = 1

In [ ]:
#Soft Encoded
soft_encoded_edge = np.zeros((uni_frq_edge.shape[0], 10))
for x in range(universal_edge.shape[0]):
    for c in range(10):
        soft_encoded_edge[x,c] = np.exp( - (np.linalg.norm(uni_frq_edge[x]- centroids_e[c], 2) ** 2) / 2)

In [ ]:
for x in soft_encoded_edge:
    x /= np.sum(x) 

In [ ]:
soft_encoded_edge[0]

In [ ]:
universal_edge.shape

In [ ]:
#Look at the edge class frequency
plt.scatter(range(10), np.sum(one_hot_encoded_edge, axis=0))

In [ ]:
np.sum(one_hot_encoded, axis=0)

In [ ]:
freq_type_bonds_edge[0]

In [ ]:
#Create edge data for each graph
x_data_edge = list()
for i in range(len(g_mol)):
    if (g_mol[i].number_of_nodes() > 10 and g_mol[i].number_of_nodes() < 100):
        v = np.zeros((soft_encoded_edge[0].shape[0]))
        #v = np.zeros((one_hot_encoded_edge[0].shape[0]))
        for n in graph_to_edges[i]:
            v += soft_encoded_edge[n]
            #v += one_hot_encoded_edge[n]
        x_data_edge.append(v)
x_data_edge = np.array(x_data_edge)
x_data_edge.shape

In [ ]:
x_data = np.hstack([x_data_node, x_data_edge])

In [ ]:
x_data.shape

In [ ]:
a = np.ones(5)

In [ ]:
x_data -= np.mean(x_data, axis=0)
x_data /= (np.max(x_data, axis=0) - np.min(x_data, axis=0))

# Classification Model #

In [ ]:
#Prepare y_data
y_data = list()
for i in range(df.shape[0]):
    if (g_mol[i].number_of_nodes() > 10 and g_mol[i].number_of_nodes() < 100):
        y_data.append(df.iloc[i]['HIV_active'])
y_data = np.array(y_data)
y_data.shape

In [ ]:
import random

In [ ]:
f = np.arange(41023)
random.Random(10).shuffle(f)
train = f[:35000]

In [ ]:
#f[35000:35010]

In [ ]:


#np.random.seed=np.random.randint(0,123456)
np.random.shuffle(train)



i_train = train[:29000]
i_val = train[29000:35000]
i_test = f[35000:]

x_train = x_data[i_train, :]
x_val = x_data[i_val, :] 

y_train = y_data[i_train]
y_val = y_data[i_val]

x_test = np.array([np.array(x_data[i,:]) for i in f[35000:]])
y_test = np.array([y_data[i] for i in f[35000:]])

In [ ]:
set(f[35000:]).intersection(set(f[:35000]))

In [ ]:
y_train.shape

In [ ]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras import optimizers

from sklearn.metrics import roc_auc_score

In [ ]:
Dropout

In [ ]:
## # define the keras model
model = Sequential()

model.add(Dense(32, activation='relu'))
model.add(Dropout(rate=0.4))
model.add(BatchNormalization())

model.add(Dense(64, activation='relu'))
model.add(Dropout(rate=0.4))
model.add(BatchNormalization())

model.add(Dense(1, activation='sigmoid'))

In [ ]:
adam = optimizers.Adam(lr=0.001)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:

model.fit(x_train, y_train, epochs=500,validation_data=(x_val, y_val), batch_size=128)

In [ ]:

# evaluate the keras model

pe, accuracy = model.evaluate(x_val, y_val)
print('Accuracy: %.2f' % (accuracy*100))



In [ ]:
p_train = model.predict(x_train)
p_val = model.predict(x_val)
p_test = model.predict(x_test)

print(" Train AUC-ROC :")
print(roc_auc_score(y_train, p_train))

print(" Valid AUC-ROC :")
print(roc_auc_score(y_val, p_val))

print(" Test AUC-ROC :")
print(roc_auc_score(y_test, p_test))

In [ ]:
"""
Script that trains multitask models on hiv dataset.
"""
from __future__ import print_function
from __future__ import division
from __future__ import unicode_literals

import numpy as np
import deepchem as dc
from deepchem.molnet import load_hiv

# Only for debug!
np.random.seed(123)

# Load hiv dataset
n_features = 512
hiv_tasks, hiv_datasets, transformers = load_hiv()
train_dataset, valid_dataset, test_dataset = hiv_datasets

# Fit models
metric = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)

transformer = dc.trans.IRVTransformer(10, len(hiv_tasks), train_dataset)
train_dataset = transformer.transform(train_dataset)
valid_dataset = transformer.transform(valid_dataset)

model = dc.models.TensorflowMultitaskIRVClassifier(
    len(hiv_tasks), K=10, batch_size=50, learning_rate=0.001)

# Fit trained model
model.fit(train_dataset)

print("Evaluating model")
train_scores = model.evaluate(train_dataset, [metric], transformers)
valid_scores = model.evaluate(valid_dataset, [metric], transformers)

print("Train scores")
print(train_scores)

print("Validation scores")
print(valid_scores)

In [ ]:
train_dataset, valid_dataset, test_dataset = hiv_datasets

In [ ]:
test_dataset = transformer.transform(test_dataset)
test_scores = model.evaluate(test_dataset, [metric], transformers)

In [ ]:
test_dataset.X.shape

In [ ]:
print(test_scores)

In [ ]:
train_dataset.ids

In [ ]:
sum(train_dataset.y)

In [ ]:
valid_dataset.ids

In [ ]:
sum(valid_dataset.y)

In [ ]:
test_dataset.X.shape

In [ ]:
len(y_test)